HAA from
https://allocatesmartly.com/hybrid-asset-allocation/


1. At the close on the last trading day of the month, measure the momentum of US Treasury Inflation-Protected Securities (TIPS, ETF: TIP).

    The strategy uses TIPS as a “canary asset” to determine whether to allocate to either an offensive or defensive asset universe. Since 1971, the momentum of simulated TIPS would have been positive about 86% of the time, meaning the vast majority of the time we would be allocating to the offensive asset universe.
    
2. If TIPS momentum is positive, select the four assets with the highest momentum from the following offensive asset universe:
    
    US large caps (represented by SPY), US small caps (IWM), developed international stocks (EFA), emerging market stocks (EEM), US real estate (VNQ), commodities (PDBC), intermediate-term US Treasuries (IEF) and long-term US Treasuries (TLT)
    
    For each of those four assets, if momentum is positive, allocate 25% of the portfolio to the asset, otherwise allocate that portion of the portfolio to cash either intermediate-term US Treasuries (IEF) or cash, depending on which has the highest momentum (US T-Bills are used as a proxy for cash momentum) (*).
    
    This is what’s known as “dual momentum”. The asset must exhibit both positive momentum (aka “time series momentum”) as well as high momentum relative to competing assets (aka “cross-sectional momentum”).
    
    Note: Commodities and treasuries are generally considered defensive assets, so the offensive portfolio will still be diversified in many cases.

3. If TIPS momentum is negative, allocate the entire portfolio to either intermediate-term US Treasuries (IEF) or cash, depending on which has the highest momentum.

4. All positions are executed at the close. Hold all positions until the last the trading day of the following month. Rebalance monthly, even if there is no change signaled.


Stratergy is closer to this implementation:

https://indexswingtrader.blogspot.com/2023/02/introducing-hybrid-asset-allocation-haa.html

Changes:
1. Momentum is calculated from the close of the second to last day of the month
    1.1 This does not change results too much (sr drops from 1.15 to 1.1, cagr increases from 0.96 to 0.97)

##### Notes:
1. adding QQQ hurts returns
2. vol scaling doesn't add a whole lot currently
    * slightly higher sr, but lower cagr

##### ToDo

1. Position sizes
    * Given an amount of equity and current prices what are the lots we should be purchasing



2. Daily Pnl / returns series
    * It looks like some sort of volatility scailing will work for improving the sharpe ratio
    * Calculate the daily pnl and then try to predict the volatility of the retruns series
    * Inputs could be the vol of the assets, with an indicator function as to whether they are in the universe


3. Drawdown series
4. Position sizing
    * Vol based position sizing
    * min corr of all assets with positive mom

##### Done

1. What are the dates?
1.1 Currently they are the month we selected the assets in, including the returns
    Thus we are attributing the returns to the wrong month
    We should lag asset selection instead of returns

In [ ]:
from datetime import datetime

import pandas as pd
import numpy as np
import yfinance as yf
import plotly.express as px

def sr(ret, N=252):
    ret = ret[ret!=0]
    print(f'SR: {np.mean(ret) / np.std(ret) * np.sqrt(N)}')

def cagr(cpnl):
    t = ((cpnl.index[-1] - cpnl.index[0]) / 365.25)
    t = (t.seconds / (24 * 60 * 60) + t.days)
    FV = cpnl.values[-1]
    print(f'CAGR: {FV ** (1/t) - 1}')

def annual_returns(df, col='ret'):
    return df.groupby(pd.Grouper(freq='Y'), group_keys=False)[col].agg(lambda x: (1+x).prod() - 1).to_frame()

In [ ]:
# offensive_universe = ['SPY', 'IWM', 'EFA', 'EEM', 'VNQ', 'DBC', 'TLT']

US_Equities = ['SPY', 'IWM'] #, 'QQQ']
Foreign_Equities = ['VEA', 'VWO']
Alternative_Assets = ['DBC', 'VNQ']
US_Bonds = ['IEF', 'TLT']

offensive_universe = US_Equities + Foreign_Equities + Alternative_Assets + US_Bonds

cash_universe = ['BIL', 'IEF']
canary_asset = ['TIP']

# #### leveraged assets
# offensive_universe = ['SSO', 'UWM', 'VEA', 'VWO', 'DBC', 'URE', 'UBT', 'UST']

# cash_universe = ['BIL', 'UST']
# canary_asset = ['TIP']


whole_universe = offensive_universe + cash_universe + canary_asset

In [ ]:
prices_df = yf.download(tickers=whole_universe,
                        end=datetime.now())


In [ ]:
prices_df = prices_df['Adj Close']

In [ ]:
prices_df

In [ ]:
px.line(prices_df.melt(ignore_index=False), y='value', color='variable', log_y=True) 

In [ ]:
class MOMENTUM():
    def __init__(self) -> None:
        pass

    def fit(self, X, y=None):
        return X
    
    def transform(self, X, y=None):
        self.calc_monthly_close(X)
        ret1m = self.get_return(1)
        ret3m = self.get_return(3)
        ret6m = self.get_return(6)
        ret12m = self.get_return(12)
        mom = (ret1m + ret3m + ret6m + ret12m) / 4
        return mom.shift(1)
    
    def calc_monthly_close(self, X):
        last_month_date = X.reset_index().groupby(pd.Grouper(key = 'Date', freq='M')).Date.apply(lambda x: x[:-1])
        self.monthly_close = X.loc[(last_month_date)].resample('M').last()
        # self.monthly_close = X.resample('M').last()
    
    def get_return(self, months):
        return self.monthly_close.pct_change(periods = months)


class MONTHLY_RETURNS():
    def __init__(self) -> None:
        pass

    def fit(self, X, y=None):
        return X
    
    def calc_monthly_close(self, X):
        self.monthly_close = X.resample('M').last()

    def transform(self, X):
        self.calc_monthly_close(X)
        return self.get_return(1)#.shift(-1)

    def get_return(self, months):
        return self.monthly_close.pct_change(periods = months)
    

class MONTHLY_PICKS():
    def __init__(self, offensive_universe, cash_universe, canary_asset) -> None:
        self.offensive_universe = offensive_universe
        self.cash_universe = cash_universe
        self.canary_asset = canary_asset
        self.regime = None
        
    def get_picks(self, X):
        self.get_regime(X)
        if self.regime:
            assets = self.get_offensive_assets(X)
        else:
            assets = self.get_defensive_assets(X)
        return assets, self.regime
    
    def get_regime(self, X):
        if X[canary_asset].values[0] > 0:
            self.regime = True
        else:
            self.regime = False

    def get_defensive_assets(self, X):
        asset = X[self.cash_universe].melt()
        # asset = asset[asset.value > 0]
        asset = asset.sort_values('value').tail(1).variable.tolist()
        return asset

    def get_offensive_assets(self, X):
        assets = X[self.offensive_universe].melt()
        assets = assets[assets.value > 0]
        assets = assets.sort_values('value').tail(4).variable.tolist()
        Cash_fraction = 4 - len(assets)
        if Cash_fraction > 0:
            defensive_asset = self.get_defensive_assets(X)
            assets = assets + defensive_asset * Cash_fraction
        return assets
    

class POSITION_SIZES():
    def __init__(self) -> None:
        pass

    def get_lots(self, capital, assets):
        #### assets is a dict of aggressive 
        pass

In [ ]:
mom = MOMENTUM()
mom_df = pd.concat([mom.transform(prices_df[sym]).to_frame() for sym in whole_universe], axis=1)

monthly_ret = MONTHLY_RETURNS()
ret_df = pd.concat([monthly_ret.transform(prices_df[sym]).to_frame() for sym in whole_universe], axis=1)


In [ ]:
px.line(mom_df.melt(ignore_index=False), y='value', color='variable')

In [ ]:
monthly_picks = MONTHLY_PICKS(offensive_universe, cash_universe, canary_asset)
monthly_assets = mom_df.dropna().groupby('Date').apply(monthly_picks.get_picks)
monthly_assets

In [ ]:
months_ret = []
months_regime = []

for Date in monthly_assets.index.unique():
    assets = monthly_assets.loc[Date][0] #.tolist()
    months_ret.append([Date, ret_df.loc[Date, assets].mean()])
    months_regime.append([Date, monthly_assets.loc[Date][1]])

strat_returns = pd.DataFrame(months_ret, columns=['Date', 'ret']).sort_values('Date').set_index('Date')
strat_regime = pd.DataFrame(months_regime, columns=['Date', 'offensive_regime']).sort_values('Date').set_index('Date')
strat_returns = pd.concat([strat_regime, strat_returns],axis=1)

strat_returns['regime_group'] = strat_returns.offensive_regime.diff().bfill().cumsum()
strat_returns['cpnl'] = (strat_returns.ret + 1).cumprod()

strat_returns['spy'] = prices_df.SPY.resample('M').last()
strat_returns['spy'] /= strat_returns.spy.values[0]
strat_returns

In [ ]:
px.bar(strat_returns, y='ret', color='offensive_regime').show()
fig = px.line(strat_returns, y='cpnl', color='offensive_regime', line_group='regime_group', log_y=True)#.show()
benchmark_trace = px.line(strat_returns, y='spy').data[0]
benchmark_trace['line']['color'] = 'black'
benchmark_trace['line']['dash'] = 'dash'
fig.add_trace(benchmark_trace)
fig.show()
px.bar(strat_returns.ret.rolling(12).std() * np.sqrt(12)).show()
px.bar(annual_returns(strat_returns, 'ret'), y='ret').show()
cagr(strat_returns.cpnl.dropna())
sr(strat_returns.ret.dropna(), N=12)

In [ ]:
cagr(strat_returns.spy.dropna())
sr(strat_returns.spy.pct_change().dropna(), N=12)

# CAGR: 0.09743790454842394
# SR: 1.0914094303126585

# CAGR: 0.09680215314678176
# SR: 1.140642818036425

In [ ]:
px.bar(strat_returns, y='ret', color='offensive_regime')

In [ ]:
vol_df = np.log(prices_df)#.apply(lambda x: np.diff(np.log(x)))
vol_df = vol_df.diff().dropna()
vol_df = (vol_df.rolling(252).std() * np.sqrt(252)).mean(axis=1).to_frame('vol')\
       + (vol_df.rolling(189).std() * np.sqrt(189)).mean(axis=1).to_frame('vol')\
       + (vol_df.rolling(126).std() * np.sqrt(126)).mean(axis=1).to_frame('vol')\
       + (vol_df.rolling(63).std() * np.sqrt(63)).mean(axis=1).to_frame('vol')
vol_df /= 4
px.bar(vol_df.resample('M').mean()).show()
vol_df = vol_df.resample('M').mean().shift(1)
                    
#                     .rolling(252).std() * np.sqrt(252)).mean(axis=1).values
# prices_df#.groupby()

In [ ]:
px.bar(strat_returns.ret.rolling(12).std() * np.sqrt(12))

In [ ]:
max_leverage = 1
vol_window_l = 12
vol_window_mm = 9
vol_window_m = 6
vol_window_s = 3

vol_target = 0.1 #25

strat_returns['rolling_vol'] = ((strat_returns.ret.rolling(vol_window_l).std() * np.sqrt(12)).shift(1)\
                                 + (strat_returns.ret.rolling(vol_window_mm).std() * np.sqrt(12)).shift(1)\
                                 + (strat_returns.ret.rolling(vol_window_m).std() * np.sqrt(12)).shift(1)\
                                 + (strat_returns.ret.rolling(vol_window_s).std() * np.sqrt(12)).shift(1)) / 4
strat_returns['leverage'] = (vol_target / strat_returns.rolling_vol).dropna()
# strat_returns['leverage'] = vol_target / vol_df
mask = strat_returns.leverage > 1
strat_returns.loc[mask, 'leverage'] = 1
strat_returns.loc[~strat_returns.offensive_regime, 'leverage'] = 1
strat_returns['ret_l'] = strat_returns.ret * strat_returns.leverage
strat_returns['cpnl_l'] = (strat_returns.ret_l + 1).cumprod()
px.bar(strat_returns, y='ret_l', color='offensive_regime').show()
fig = px.line(strat_returns, y='cpnl_l', color='offensive_regime', line_group='regime_group', log_y=True)#.show()
benchmark_trace = px.line(strat_returns, y='spy').data[0]
benchmark_trace['line']['color'] = 'black'
benchmark_trace['line']['dash'] = 'dash'
fig.add_trace(benchmark_trace)
fig.show()
px.bar(strat_returns, y='leverage', color='offensive_regime').show()
px.bar(strat_returns.ret_l.rolling(12).std() * np.sqrt(12)).show()
px.bar(annual_returns(strat_returns, 'ret_l'), y='ret_l').show()
cagr(strat_returns.cpnl_l.dropna())
sr(strat_returns.ret_l.dropna(), N=12)